In [32]:
import pandas as pd
import numpy as np
import requests


In [45]:
df_occupations = pd.read_csv('data/ESCO/occupations_en.csv')

In [46]:
df_occupations.head()

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,code
0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2654,technical director,technical and operations director\nhead of tec...,NaN,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Technical directors realise the artistic visio...,2654.1.7
1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,metal drawing machine operator,metal drawing machine operator\nmetal drawing ...,NaN,released,2016-07-05T17:09:43Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Metal drawing machine operators set up and ope...,8121.4
2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,precision device inspector,inspector of precision instruments\nprecision ...,NaN,released,2016-07-06T09:21:20Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Precision device inspectors make sure precisio...,7543.10.3
3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,air traffic safety technician,air traffic safety electronics hardware specia...,NaN,released,2017-01-17T11:40:37Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Air traffic safety technicians provide technic...,3155.1
4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,hospitality revenue manager,hospitality revenues manager\nyield manager\nh...,NaN,released,2017-01-17T13:33:42Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Hospitality revenue managers maximise revenue ...,2431.9


In [47]:
df_occupations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conceptType              3008 non-null   object
 1   conceptUri               3008 non-null   object
 2   iscoGroup                3008 non-null   int64 
 3   preferredLabel           3008 non-null   object
 4   altLabels                3007 non-null   object
 5   hiddenLabels             6 non-null      object
 6   status                   3008 non-null   object
 7   modifiedDate             3008 non-null   object
 8   regulatedProfessionNote  3008 non-null   object
 9   scopeNote                304 non-null    object
 10  definition               8 non-null      object
 11  inScheme                 3008 non-null   object
 12  description              3008 non-null   object
 13  code                     3008 non-null   object
dtypes: int64(1), object(13)
memory usage: 32

In [49]:
role = 'pig breeder'

In [78]:
# get uri for selected role

role_extract = df_occupations[df_occupations['preferredLabel'] == role]
role_uri = role_extract['conceptUri']


[2777    http://data.europa.eu/esco/occupation/ea3db1b3...
Name: conceptUri, dtype: object]


In [86]:
# search for all jobs that have 'manager' in their title  --- ignore this cell for now
url_title = "https://ec.europa.eu/esco/api/search"

params = {
    'language': 'en',
    'type': 'occupation',
    'text': 'pig'
         }

api_title = requests.get(url=url, params=params).json()

In [99]:
# get role information from the ESCO API

url = "https://ec.europa.eu/esco/api/resource/skill"

params = {
    'uri': role_uri,
    'language': 'en',
         }

api_skills = requests.get(url=url, params=params).json()

# this line gets you the whole API url; easier to read on your browser
requests.get(url=url, params=params).url


'https://ec.europa.eu/esco/api/resource/skill?uri=http%3A%2F%2Fdata.europa.eu%2Fesco%2Foccupation%2Fea3db1b3-7c03-4306-9e3d-7e20eb7f1826&language=en'

In [109]:
# gets a list of all skill descriptions and extracts the skill title only

api_skills_dict = api_skills.get('_links').get('hasEssentialSkill')



In [110]:
api_skills_dict

[{'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/7837ef5c-2c81-4a89-8227-b4d4f1f7331e&language=en',
  'uri': 'http://data.europa.eu/esco/skill/7837ef5c-2c81-4a89-8227-b4d4f1f7331e',
  'title': 'maintain professional records',
  'skillType': 'http://data.europa.eu/esco/skill-type/skill'},
 {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/80c4d5a3-fe4a-4f6f-9bb0-1f6605f98d43&language=en',
  'uri': 'http://data.europa.eu/esco/skill/80c4d5a3-fe4a-4f6f-9bb0-1f6605f98d43',
  'title': 'create animal records',
  'skillType': 'http://data.europa.eu/esco/skill-type/skill'},
 {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/2879557e-7c79-4f58-b076-99505ef34918&language=en',
  'uri': 'http://data.europa.eu/esco/skill/2879557e-7c79-4f58-b076-99505ef34918',
  'title': 'carry out specialised procedures for pigs',
  'skillType': 'http://data.europa.eu/esco/skill-type/skill'

In [113]:
skills_list = [skill.get('title') for skill in api_skills_dict]

In [114]:
skills_list

['maintain professional records',
 'create animal records',
 'carry out specialised procedures for pigs',
 'manage animal hygiene',
 'livestock feeding',
 'manage animal biosecurity',
 'administer treatment to animals',
 'monitor livestock',
 'care for juvenile animals',
 'maintain animal accommodation',
 'manage the health and welfare of livestock',
 'provide nutrition to animals',
 'animal nutrition',
 'control animal movement',
 'assist in transportation of animals',
 'select livestock',
 'manage livestock',
 'animal welfare legislation',
 'assist animal birth',
 'feed livestock',
 'health and safety regulations',
 'administer specific drugs to facilitate breeding',
 'signs of animal illness',
 'monitor the welfare of animals',
 'livestock species',
 'dispose of dead animals',
 'provide first aid to animals',
 'livestock reproduction',
 'operate farm equipment',
 'manage pig breeding']

In [119]:
### run the whole thing ###

# select role
role = 'food safety inspector'

# get uri for selected role
role_extract = df_occupations[df_occupations['preferredLabel'] == role]
role_uri = role_extract['conceptUri']

# get role information from the ESCO API
url = "https://ec.europa.eu/esco/api/resource/skill"
params = {
    'uri': role_uri,
    'language': 'en',
         }
api_skills = requests.get(url=url, params=params).json()

# this line gets you the whole API url; easier to read on your browser
requests.get(url=url, params=params).url

# gets a list of all skill descriptions and extracts the skill title only
api_skills_dict = api_skills.get('_links').get('hasEssentialSkill')
skills_list = [skill.get('title') for skill in api_skills_dict]

skills_list

['perform quality audits',
 'food toxicity',
 'foodborne diseases',
 'identify the factors causing changes in food during storage',
 'assess food samples',
 'risks associated to physical, chemical, biological hazards in food and beverages',
 'work in cold environments',
 'manage communications with food industry governmental bodies',
 'analyse samples of food and beverages',
 'identify hazards in the workplace',
 'european food safety policy',
 'assess HACCP implementation in plants',
 'food and beverage industry',
 'legislation about animal origin products',
 'quality assurance methodologies',
 'apply HACCP',
 'keep up-to-date with regulations',
 'manage health and safety standards',
 'cultural practices regarding animal slaughter',
 'maintain relationships with government agencies',
 'health, safety and hygiene legislation',
 'write routine reports',
 'food safety standards',
 'laboratory-based sciences',
 'lead inspections',
 'apply requirements concerning manufacturing of food and 